# Product enquiry vs Chatbot

In [1]:
# pip install pymorphy2

In [3]:
import os
import string
import annoy
import json
import pickle

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import KeyedVectors


import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [4]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [10]:
# Upload vectorizer
with open('../input/vectoriver-title/vectorizer_title.pk' , 'rb') as f:
    vectorizer = pickle.load(f)

In [6]:
# Upload ML model
with open('../input/pickled-model-svm/svm_model_pkl' , 'rb') as f:
    model = pickle.load(f)

In [34]:
index = annoy.AnnoyIndex(100, metric='angular')
#index.load('../input/final-chatbot/speaker.ann')

#index_prod = annoy.AnnoyIndex(100, metric='angular')
#index_prod.load('../input/final-products/index_prod.ann')

In [38]:
# Upload word vectors

word_vectors_chat = KeyedVectors.load("../input/final-chatbot-word2vec/word2vec.wordvectors", mmap='r')
keys_chat = word_vectors.key_to_index

word_vectors_prod = KeyedVectors.load("../input/final-products/word2vec_prod.wordvectors", mmap='r')
keys_prod = word_vectors_prod.key_to_index

In [39]:
# Upload index maps

with open('../input/final-chatbot/index_map.json') as json_file:
    index_map_chat = json.load(json_file)

with open('../input/final-products/index_map_prod.json') as json_file:
    index_map_prod = json.load(json_file)

In [40]:
def get_answer(question):
    
    word_vectors = word_vectors_chat
    keys = keys_chat
    index.load('../input/final-chatbot/speaker.ann')
    index_map = index_map_chat
    
    preprocessed_question = preprocess_txt(question)
    preprocessed_question_vec = vectorizer.transform([' '.join(preprocessed_question)])  
    
    prediction = model.predict(preprocessed_question_vec)
    

    
    if prediction == 1:
        
        word_vectors = word_vectors_prod
        keys = keys_prod
        index.load('../input/final-products/index_prod.ann')
        index_map = index_map_prod

    n_w2v = 0
    vector = np.zeros(100)
    for word in preprocessed_question:
        if word in keys.keys():
            vector += word_vectors[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    answer_index = index.get_nns_by_vector(vector, 1)
    return index_map[str(answer_index[0])]        

In [15]:

assert(get_answer('Юбка детская ORBY').startswith('58e3cfe6132ca50e053f5f82'))

In [41]:
get_answer('Юбка детская ORBY')

'58e3cfe6132ca50e053f5f82 Юбка детская ORBY'

In [16]:
assert(not get_answer('Где ключи от танка').startswith('5')) 

In [42]:
get_answer('Где ключи от танка')

'у миня мод есть на прокачку, двойной. классный. \n'